<a href="https://colab.research.google.com/github/draculess99/Tensorflow2/blob/master/Section12_TensorFlow_Serving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2343  100  2343    0     0  11773      0 --:--:-- --:--:-- --:--:-- 11833
OK


In [2]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package tensorflow-model-server2


In [74]:
%%bash 
ps -aux

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  39192  6460 ?        Ss   05:24   0:00 /bin/bash -e /datalab/run.sh
root           8  0.2  0.5 709464 79728 ?        Sl   05:24   0:15 /tools/node/bin/node /datalab/web/app.js
root          24  0.5  0.7 409312 105500 ?       Sl   05:24   0:43 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         122  0.0  0.0  35884  4872 ?        Ss   05:25   0:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/drive_fs.txt
root       10046 24.5 22.2 42731344 2971908 ?    Ssl  07:14   4:12 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-e9e57f8a-06b5-49f9-8c11-99576b367997.json
root       12735  0.0  0.0  39196  6548 ?        S    07:31   0:00 bash
root       12736  0.0  0.0  59028  6280 ?        R    07:31   0:00 ps -aux


In [76]:
!apt-get update & apt-get install tensorflow-model-server

Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Err:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
  The following signatures were invalid: EXPKEYSIG 544B7F63BF9E4D5F Tensorflow Serving Developer (Tensorflow Serving APT repository key) <tensorflow-serving-dev@googlegroups.com>
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Reading package lists... Done
Ign:10 https://developer.downl

In [77]:
%%bash 
ps -aux

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  39192  6460 ?        Ss   05:24   0:00 /bin/bash -e /datalab/run.sh
root           8  0.2  0.5 709464 79724 ?        Sl   05:24   0:15 /tools/node/bin/node /datalab/web/app.js
root          24  0.5  0.7 409312 105532 ?       Sl   05:24   0:44 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         122  0.0  0.0  35884  4872 ?        Ss   05:25   0:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/drive_fs.txt
root       10046 23.5 22.2 42731344 2971908 ?    Ssl  07:14   4:12 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-e9e57f8a-06b5-49f9-8c11-99576b367997.json
root       13465  3.0  0.0  39192  6556 ?        S    07:31   0:00 bash
root       13466  0.0  0.0  59032  6288 ?        R    07:31   0:00 ps -aux


In [6]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 45kB/s 
     |████████████████████████████████| 3.2MB 33.5MB/s 
     |████████████████████████████████| 368kB 42.6MB/s 
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0


In [7]:
!pip install requests

In [8]:
import os
import json
import random
import requests
import subprocess
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10

%matplotlib inline
tf.__version__

'1.13.1'

## Stage 2: Import project dependencies

In [9]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


## Stage 3: Dataset preprocessing

In [0]:
# ### Loading the dataset

In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [0]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [0]:
### Image normalization

In [0]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [15]:
X_train.shape

(50000, 32, 32, 3)

In [0]:
## Stage 4: Defining the model

# NOTE: We are using the model from the Section for Convolutional neural networks

## Stage 4: Defining the model

NOTE: We are using the model from the Section for Convolutional neural networks

In [17]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[32, 32, 3]))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
### Compiling the model


In [0]:
model.compile(optimizer='Adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['sparse_categorical_accuracy'])

In [0]:
### Training the model

In [36]:
model.fit(X_train, 
          y_train, 
          batch_size=128, 
          epochs=10)

Epoch 1/10
50000/50000 [==============================] - 11s 229us/sample - loss: 0.2503 - sparse_categorical_accuracy: 0.9116
Epoch 2/10
50000/50000 [==============================] - 11s 225us/sample - loss: 0.2097 - sparse_categorical_accuracy: 0.9265
Epoch 3/10
50000/50000 [==============================] - 11s 224us/sample - loss: 0.1672 - sparse_categorical_accuracy: 0.9412
Epoch 4/10
50000/50000 [==============================] - 11s 224us/sample - loss: 0.1380 - sparse_categorical_accuracy: 0.9510
Epoch 5/10
50000/50000 [==============================] - 11s 223us/sample - loss: 0.1166 - sparse_categorical_accuracy: 0.9585
Epoch 6/10
50000/50000 [==============================] - 11s 223us/sample - loss: 0.1089 - sparse_categorical_accuracy: 0.9604
Epoch 7/10
50000/50000 [==============================] - 11s 223us/sample - loss: 0.0880 - sparse_categorical_accuracy: 0.9696
Epoch 8/10
50000/50000 [==============================] - 11s 223us/sample - loss: 0.0843 - sparse_categ

In [0]:
### Model evaluation

In [38]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 143us/sample - loss: 1.6855 - sparse_categorical_accuracy: 0.7267


In [53]:
print("Test accuracy is {}".format(test_accuracy))

Test accuracy is 0.7267000079154968


## Stage 5: Saving the model for production

In [0]:
### Creating the directory for the model

In [0]:
MODEL_DIR = "model/"
version = 1

In [0]:
export_path = os.path.join(MODEL_DIR, str(version))
export_path

In [0]:
if os.path.isdir(export_path):
    !rm -r {export_path}

In [0]:
### Saving the model for the TensorFlow Serving

In [57]:
tf.saved_model.simple_save(tf.keras.backend.get_session(), export_dir=export_path, inputs={"input_image":model.input}, outputs={t.name:t for t in model.outputs})

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: model/1/saved_model.pb


## Stage 6: Setting up the production environment

In [0]:
### Exporting the MODEL_DIR to environment variables

In [0]:
os.environ['MODEL_DIR'] = os.path.abspath(MODEL_DIR)

In [0]:
### Running the TensorFlow Serving REST API

In [71]:
%%bash --bg
nohup tensorflow_model_server --rest_api_port=8501 --model_name=cifar10 --model_base_path="${MODEL_DIR}" >server5.log 2>&1

Starting job # 4 in a separate thread.


In [72]:
!tail server5.log

nohup: failed to run command 'tensorflow_model_server': No such file or directory


In [70]:
%%bash 
ps -aux

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  39192  6460 ?        Ss   05:24   0:00 /bin/bash -e /datalab/run.sh
root           8  0.2  0.5 709464 79724 ?        Sl   05:24   0:15 /tools/node/bin/node /datalab/web/app.js
root          24  0.5  0.7 409312 105500 ?       Sl   05:24   0:43 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         122  0.0  0.0  35884  4872 ?        Ss   05:25   0:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/drive_fs.txt
root       10046 29.1 22.2 42731344 2971908 ?    Ssl  07:14   4:11 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-e9e57f8a-06b5-49f9-8c11-99576b367997.json
root       12681  0.0  0.0  39192  6592 ?        S    07:28   0:00 bash
root       12682  0.0  0.0  59028  6532 ?        R    07:28   0:00 ps -aux
